In [3]:
import pandas as pd
import numpy as np

In [4]:
video_games_sales=pd.read_csv("Video_Games_Sales_as_at_22_Dec_2016.csv")

In [12]:
v2 = video_games_sales.drop(columns=['User_Score', 'NA_Sales', 'JP_Sales', 'Other_Sales', 'Year_of_Release', 'EU_Sales'])

# Checking the shape of the cleaned DataFrame
cleaned_shape = v2.shape
cleaned_shape

(16719, 10)

In [16]:
##################TRAIN TEST SPLIT########################
from sklearn.model_selection import train_test_split

#  target  'Global_Sales' 
X = v2.drop(columns=['Global_Sales'])  
y = v2['Global_Sales']  

#  split 80-20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"train{X_train.shape}, {y_train.shape}")
print(f"test {X_test.shape}, {y_test.shape}")

train(13375, 9), (13375,)
test (3344, 9), (3344,)


In [36]:
###################  FYLLNING AV TOMMA CELLER#######################
# median och mode ifyllnad X_train 
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

X_train2 = X_train.copy()
X_train2[numeric_cols] = X_train2[numeric_cols].fillna(X_train2[numeric_cols].median())


categorical_cols = X_train.select_dtypes(include=['object']).columns
X_train2[categorical_cols] = X_train2[categorical_cols].fillna(X_train2[categorical_cols].mode().iloc[0])


In [39]:
# median och mode ifyllnad X_test  (annan median o mode)
numeric_cols = X_test.select_dtypes(include=['float64', 'int64']).columns

X_test2 = X_test.copy()
X_test2[numeric_cols] = X_test2[numeric_cols].fillna(X_test2[numeric_cols].median())


categorical_cols = X_test.select_dtypes(include=['object']).columns
X_test2[categorical_cols] = X_test2[categorical_cols].fillna(X_test2[categorical_cols].mode().iloc[0])
########################################################

In [ ]:
#############   ENCODINGS #########################

In [43]:
X_train3 = X_train2.copy()

# Create one-hot encoded columns for 'Ubisoft', 'Nintendo', and 'The rest'
X_train3['Dev_Ubisoft'] = X_train3['Developer'].apply(lambda x: 1 if x == 'Ubisoft' else 0)
X_train3['Dev_Nintendo'] = X_train3['Developer'].apply(lambda x: 1 if x == 'Nintendo' else 0)

# Create 'The rest' column (1 where Developer is not Ubisoft or Nintendo)
X_train3['Dev_rest'] = X_train3.apply(lambda row: 1 if (row['Developer'] != 'Ubisoft' and row['Developer'] != 'Nintendo') else 0, axis=1)


# Drop the original 'Developer' column
X_train3 = X_train3.drop(columns=['Developer'])

In [45]:
X_test3 = X_test2.copy()

# Create one-hot encoded columns for 'Ubisoft', 'Nintendo', and 'The rest'
X_test3['Dev_Ubisoft'] = X_test3['Developer'].apply(lambda x: 1 if x == 'Ubisoft' else 0)
X_test3['Dev_Nintendo'] = X_test3['Developer'].apply(lambda x: 1 if x == 'Nintendo' else 0)

# Create 'The rest' column (1 where Developer is not Ubisoft or Nintendo)
X_test3['Dev_rest'] = X_test3.apply(lambda row: 1 if (row['Developer'] != 'Ubisoft' and row['Developer'] != 'Nintendo') else 0, axis=1)


# Drop the original 'Developer' column
X_test3 = X_test3.drop(columns=['Developer'])

In [47]:
# one-hot encoding 'E' (Everyone) and Rest
X_train3['Rating_E'] = X_train3['Rating'].apply(lambda x: 1 if x == 'E' else 0)

#Rest column (1 where Rating is not 'E')
X_train3['Rating_Rest'] = X_train3['Rating'].apply(lambda x: 1 if x != 'E' else 0)

# Drop the old Rating column
X_train3 = X_train3.drop(columns=['Rating'])


In [49]:
# one-hot encoding 'E' (Everyone) and Rest
X_test3['Rating_E'] = X_test3['Rating'].apply(lambda x: 1 if x == 'E' else 0)

#Rest column (1 where Rating is not 'E')
X_test3['Rating_Rest'] = X_test3['Rating'].apply(lambda x: 1 if x != 'E' else 0)

# Drop the old Rating column
X_test3 = X_test3.drop(columns=['Rating'])

In [ ]:
#############   ENCODINGS STOP #########################

In [50]:
X_test3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3344 entries, 6991 to 12077
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          3344 non-null   object 
 1   Platform      3344 non-null   object 
 2   Genre         3344 non-null   object 
 3   Publisher     3344 non-null   object 
 4   Critic_Score  3344 non-null   float64
 5   Critic_Count  3344 non-null   float64
 6   User_Count    3344 non-null   float64
 7   Dev_Ubisoft   3344 non-null   int64  
 8   Dev_Nintendo  3344 non-null   int64  
 9   Dev_rest      3344 non-null   int64  
 10  Rating_E      3344 non-null   int64  
 11  Rating_Rest   3344 non-null   int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 339.6+ KB


In [51]:
from sklearn.linear_model import LinearRegression

In [64]:
X_train_selected = X_train3.iloc[:, -8:]  # Select last 3 columns for example
model = LinearRegression()
model.fit(X_train_selected, y_train)

LinearRegression()

In [65]:
coef = model.coef_
inte = model.intercept_

In [66]:
print(coef)

[ 1.27617135e-02  1.59914836e-02  6.90915202e-04 -1.50456164e+00
  2.92808081e+00 -1.42351917e+00  6.64767260e-02 -6.64767260e-02]


In [68]:
X_test_selected = X_test3.iloc[:, -8:]  # Select the same columns as in training

# Step 8: Make predictions using the fitted model
predictions = model.predict(X_test_selected)

# Print predictions
print(predictions)

[0.96346622 0.45106967 0.52313025 ... 0.45106967 0.21737668 0.45106967]


In [ ]:
#############   whole process###############################

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Sample DataFrame
data = {
    'Platform': ['PS2', 'X360', 'Wii', 'PS3', 'DS', 'PS2'],
    'Genre': ['Action', 'Adventure', 'Puzzle', 'Action', 'Puzzle', 'Action'],
    'Rating': ['E', 'M', 'E', None, 'T', 'E'],
    'Global_Sales': [3.5, 5.2, 1.3, 4.1, 2.2, 3.0]
}
df = pd.DataFrame(data)

# Step 1: Split the data into features and target variable
X = df.drop(columns=['Global_Sales'])
y = df['Global_Sales']

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Handle missing values, e.g., with SimpleImputer
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Convert back to DataFrame for easier column selection
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns)

# Step 4: One-Hot Encode categorical variables for X_train
X_train_encoded = pd.get_dummies(X_train_imputed, drop_first=True)

# Step 5: One-Hot Encode categorical variables for X_test
X_test_encoded = pd.get_dummies(X_test_imputed, drop_first=True)

# Align columns in train and test sets
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

# Step 6: Select the last columns (e.g., all columns except the first few)
# This assumes you want to drop the first columns and keep the last columns for model fitting.
# Adjust the indexing according to your needs.
X_train_selected = X_train_encoded.iloc[:, -3:]  # Select last 3 columns for example

# Step 7: Fit the model using the selected features
model = LinearRegression()
model.fit(X_train_selected, y_train)

# Now you can also select the corresponding columns in X_test_encoded for predictions
X_test_selected = X_test_encoded.iloc[:, -3:]  # Select the same columns as in training

# Step 8: Make predictions using the fitted model
predictions = model.predict(X_test_selected)

# Print predictions
print(predictions)
